#### Marker auto-detection
#### Text preprocessing

#### Importing the modules:

In [139]:
# Data loading and plotting packages
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib

In [140]:
# Model metric packages 
#from sklearn import cross_validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,roc_curve, auc
from sklearn import metrics 

In [141]:
# Text pre-processing packages 
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import time 

#### Loading the root data into the system:

In [142]:
root_data=pd.read_csv('E:\All accumulated folder\FREELANCE\clean data\Assesment File_v1.csv', encoding = 'unicode_escape')
root_data.shape

(308, 2)

In [143]:
root_data.head()

,Conversations,Sub_markers
0,I acknowledge the progress we made in this jou...,8.9
1,I acknowledge the progress I made in this jour...,8.9
2,I would like to appreciate your exploration to...,8.9
3,How would you like acknowledge yourself before...,8.9
4,If we have covered all you wanted to achieve a...,8.8


In [144]:
set(root_data)

root_data["Sub_markers"].value_counts()

3.1    69
8.3    40
8.2    30
8.1    26
7.2    16
6.6    13
8.6    10
7.1    10
7.7     9
8.5     9
5.4     7
7.5     6
7.8     6
7.4     5
8.7     5
7.3     5
5.2     5
8.4     5
7.6     5
8.9     4
4.3     4
4.2     4
6.7     4
8.8     3
5.5     3
4.4     2
7.9     1
6.8     1
Name: Sub_markers, dtype: int64

In [145]:
# Removing the missing values from free format description

root_data=root_data[root_data["Conversations"].notnull()]

In [146]:
root_data = root_data.astype(object)

In [147]:
# # Removing records which has srong words  in "Free format Desc" column

# #Strong words-If these words are present then it is a marker word:

# strong_words=['Coach', 'client', 'accomplish' 'session' ]

# # Removing the strong word records in "Free format Desc" column
# root_data1=root_data[~root_data['Sub_markers'].str.contains('|'.join(strong_words),case=False)]
# print("Number of records removed based on strong words are ",root_data.shape[0]-root_data1.shape[0])

# # Reset and drop the index 
# root_data1.reset_index(inplace=True,drop=True)

# # Result:Number of records removed based on strong words are  137

#### -----------------------------------------------------------------Text Data  Pre-processing-----------------------------------------------------------------------------------------

##### Step 1: Removal of punctuations and special characteristics and handling case senitivity 

In [148]:
#Removal of Special Characters and Punctuations 
root_data['Free_format_Desc'] = root_data['Conversations'] .str.replace('[^\w\s]','')

# converting into lower case 
root_data['Free_format_Desc'] = root_data['Free_format_Desc'].apply(lambda x: " ".join(x.lower() for x in x.split()))

##### Step 2: Elaborating Abbrevations

In [149]:
# class for replace the Abbrevations and spell corrections using word map 
class WordReplacer(object): 
 
    def __init__(self, word_map):
            self.word_map = word_map 
    def replace(self, word): 
         return self.word_map.get(word, word)

In [150]:
# function to replace abbrevations and spell corrections 
def replace_text(text):
    tokenized_text = word_tokenize(text.lower()) 
    replaced_text=[replacer.replace(word) for word in tokenized_text ]
    replaced_text =' '.join(replaced_text)
    return replaced_text

In [151]:
# # word map to handle all the abbrevations and spell corrections that occured in metrics in textual data 
# wordmap={'What do you' : 'What do you want',
#          'we want' : 'What do we want',
#          'seek' : 'What do you seek',
#          'desire' : 'What do you desire',
#          'work on' : 'What do you work',
#          'talk about' : 'What do you want to talk about',
#          'is on your mind to discuss to accomplish' : 'What do you want to discuss to accomplish',
#          'achieve' : 'What do you want achieve',
#          'aim for' : 'What do you aim for',
#          'look forward to as an outcome or goal in the session' : 'What do you want to look forward as an outcome or goal in the session'
# # Abbrevations labelling
    
# }


In [152]:
# # Replacing observations and spell corrections on "Free_format_updated" column
# replacer=WordReplacer(wordmap)
# root_data1['Free_format_Desc']=root_data1['Free_format_Desc'].apply(replace_text)

In [153]:
# # Manual corrections of words which occured in metrics created from text data 
# root_data1['Free_format_Desc_Updated'] = root_data1['Free_format_Desc_Updated'] .str.replace()

##### Step 3: Tokenazation and Removal of Stopwords 

In [154]:
# import nltk
# nltk.download()

In [155]:
# Sysytem defined stop-words-(Eliminated all negated stopwords(Eg: off,over,doesn't,not etc.,))
#sample structure
stop_words=stopwords.words('english')
rm_stop=['before', 'after', 'above', 'below', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'no',
         'nor', 'not', 'too', 'very', "don't", "aren't", "couldn't", 'didn', "didn't", 'doesn', "doesn't", "hadn't", "hasn't",
          "haven't", "wasn't", "weren't"]
stop_words= [word for word in stop_words if word  not in rm_stop]

# user defined stopwords(based on 1 gram analysis)
stop_words.extend([#'number', 'plate','paint','presume','pour','car','vehicle','tp','policyholder','policy holder','ph','phv','insured','policy','holder',
                   #stopwords from unigrams
                   #'tbc','go','ab','tl','ncrc','ahs','fr','cv','nar','ne','xs','lr',

                   # Stopwords taken from TFIDF 
'a','also','although','always','am','among','amongst','amoungst','amount','an','and','another','any','anyhow','anyone',
'anything','anyway','anywhere','are','around','as','at','be','became','because','become','becomes','becoming','been','being',
'between','bill','but','by','call','cry','de','describe','due','during','each','eg','either','eleven','else','even','ever',
'fifteen','fifty','for','former','formerly','forty','from','further','give','go','here','hereafter','hereby','herein','hereupon',
'hers','him','his','how','however','i','ie','if','inc','interest','is','it','its','itself','latter','latterly','ltd','made',
'may','me','meanwhile','might','mill','mine','my','myself','name','namely','nine','no','noone','now','often','once','our',
'perhaps','rather','re','same','see','seem','seemed','seeming','seems','she','show','since','sincere','six','sixty','so',
'sometime','sometimes','somewhere','still','such','system','take','ten','that','the','their','them','themselves','then',
'thence','there','thereafter','thereby','therefore','therein','thereupon','these','they','third','this','those','thus','twelve',
'twenty','un','until','us','was','we','were','what','whatever','when','whence','whenever','where','whereafter','whereas','whereby',
'wherein','whereupon','wherever','whether','which','who','whoever','whom','whose','why','will','would','you','your','yours',
'yourself','yourselves','in'])

print("Total number of customized stopwords are :",len(stop_words))

# Result: Total number of customized stopwords are : 337

Total number of customized stopwords are : 323


In [156]:
# Function to replace stopwords 
def clean_text(text):
   
    # Tokenization of text
    tokenized_text = word_tokenize(text.lower()) 
    
    # Removal of Stop words 
    cleaned_text =[t for t in tokenized_text if t not in stop_words and re.match('[a-zA-Z\\-][a-zA-Z\\-]{1,}', t) and t.isalpha()]
    #Join all words
    cleaned_text =' '.join(cleaned_text)
    return cleaned_text

In [157]:
# Replacing Stopwords in "Free_format_updated" column
root_data['Free_format_Desc']=root_data['Free_format_Desc'].apply(clean_text)

In [158]:
# Text before stopword removal and handling abbrevations 
root_data['Free_format_Desc'].head(10)

0                        acknowledge progress journey
1                        acknowledge progress journey
2    like appreciate exploration aware needed achieve
3                       like acknowledge before close
4          covered wanted achieve ready close session
5            covered wanted today ready close session
6                                 ready close session
7                                 people turn support
8                           coming up helpful actions
9                             issues need help action
Name: Free_format_Desc, dtype: object

##### Step 4: Lemmatization of words 

In [159]:
# Function to perform Lematization
lmtzr = WordNetLemmatizer()
def clean_text_lemma(text):
        
    # Tokenization of text
    tokenized_text = word_tokenize(text.lower()) 
    
    # Lemmatization
    cleaned_text =[lmtzr.lemmatize(t,pos='v') for t in tokenized_text]
    
    #Joining all words
    cleaned_text =' '.join(cleaned_text)
    return cleaned_text

In [129]:
# Performing lematization on "Free_format_updated" column
root_data['Free_format_Desc']=root_data['Free_format_Desc'].apply(clean_text_lemma)

In [130]:
root_data['Free_format_Desc'].head(3)

0                      acknowledge progress journey
1                      acknowledge progress journey
2    like appreciate exploration aware need achieve
Name: Free_format_Desc, dtype: object

In [131]:
root_data

,Conversations,Sub_markers,Free_format_Desc
0,I acknowledge the progress we made in this jou...,8.9,acknowledge progress journey
1,I acknowledge the progress I made in this jour...,8.9,acknowledge progress journey
2,I would like to appreciate your exploration to...,8.9,like appreciate exploration aware need achieve
3,How would you like acknowledge yourself before...,8.9,like acknowledge before close
4,If we have covered all you wanted to achieve a...,8.8,cover want achieve ready close session
5,If we have covered all you wanted to do today ...,8.8,cover want today ready close session
6,Are ready to close the session?,8.8,ready close session
7,Who may the be the people you can turn to for ...,8.6,people turn support
8,What else is coming up as being helpful for yo...,8.6,come up helpful action
9,What do you see as issues you may need help wi...,8.6,issue need help action


##### Step 5 :Removing the Records of strong words 

In [132]:
# # # Removing the strong word records in "free flow text" column
# root_data=root_data[~root_data['Free_format_Desc'].str.contains('|'.join(strong_words),case=False)]
# root_data.reset_index(inplace=True,drop=True)

#### -----------------------------------------------------------------------End of Text Data  Pre-Processing------------------------------------------------------------

In [137]:
X = root_data.drop("Sub_markers",axis=1)
y = root_data["Sub_markers"]
X_train, X_val, y_train, y_val=train_test_split(X,y,test_size=.30,random_state=0)
# X_train.reset_index(inplace=True,drop=True)
# X_val.reset_index(inplace=True,drop=True)
# y_train.reset_index(inplace=True,drop=True)
# y_val.reset_index(inplace=True,drop=True)

# # from sklearn.model_selection import train_test_split
# X = root_data.drop("Sub_markers",axis=1)
# y = root_data["Sub_markers"]
# X, X_test, y, y_test = train_test_split(X_data, y_data, test_size=0.2, train_size=0.8)
# X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.25,train_size =0.75)
# X_train[:1]

In [138]:
y_train

241    3.1
237    3.1
167    7.4
173    7.5
291    4.2
218    3.1
145    7.1
261    3.1
255    3.1
76     8.3
156    7.2
184    7.7
146    7.2
166    7.3
289    4.3
44     8.1
182    7.6
1      8.9
205    NaN
111    8.2
288    4.3
171    7.4
120    8.2
54     8.1
269    3.1
18     8.7
209    5.4
79     8.3
159    7.2
56     8.1
      ... 
147    7.2
283    3.1
177    7.5
99     8.3
197    7.8
243    3.1
115    8.2
265    3.1
72     8.3
25     8.5
165    7.3
294    4.2
174    7.5
296    6.6
39     8.1
193    7.8
88     8.3
70     8.3
87     8.3
292    4.2
242    3.1
277    3.1
211    5.4
9      8.6
195    7.8
251    3.1
192    7.8
117    8.2
47     8.1
172    7.5
Name: Sub_markers, Length: 215, dtype: object

In [83]:
# LoadingCount vectorizer packages 
from sklearn.feature_extraction.text import TfidfVectorizer

In [84]:
# # creating object for the TFIDF
# vectorizer_tfidf_2_3_gram = TfidfVectorizer(
#         ngram_range=(2,3),  # features made of a single tokens
#     use_idf=True,  # enable inverse-document-frequency reweighting
#     smooth_idf=True,  # prevents zero division for unseen words
#     sublinear_tf=False)

In [85]:
# # Fitting Count Vectoriser on the data 
# vectorizer_tfidf_2_3_gram.fit(root_data['Free_format_Desc'])

# #Transforming on training data 
# tfidf_2_3_gram=vectorizer_tfidf_2_3_gram.transform(X_train['Free_format_Desc'])

In [86]:
# #loading SelectKbest package 
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2,f_classif,mutual_info_classif

# # Creating object for the selectKbest 
# kselect_2_3_gram=SelectKBest(chi2,k=400)

# #Fitting on SelectKbest on Training data
# kselect_2_3_gram.fit(tfidf_2_3_gram,y_train)

# # #Transforming on training dataset 
# # tfidf_kbest_2_3_gram=kselect_2_3_gram.transform(tfidf_2_3_gram)


In [87]:
# Dictionary of all words from train corpus with their counts.
words_counts = {}
for comments in X_train:
    for word in comments.split():
        if word not in words_counts:
            words_counts[word] = 1
        words_counts[word] += 1
        
DICT_SIZE = 10000
POPULAR_WORDS = sorted(words_counts, key=words_counts.get, reverse=True)[:DICT_SIZE]
WORDS_TO_INDEX = {key: rank for rank, key in enumerate(POPULAR_WORDS, 0)}
INDEX_TO_WORDS = {index:word for word, index in WORDS_TO_INDEX.items()}
ALL_WORDS = WORDS_TO_INDEX.keys()

In [88]:
# Lets take a look at top 10 popular words
POPULAR_WORDS[:10]

['Conversations', 'Free_format_Desc']

In [89]:
from scipy import sparse as sp_sparse

In [90]:
def my_bag_of_words(text, words_to_index, dict_size):
    """
        text: a string
        dict_size: size of the dictionary
        
        return a vector which is a bag-of-words representation of 'text'
    """
    result_vector = np.zeros(dict_size)
    for word in text.split(' '):
        if word in words_to_index:
            result_vector[words_to_index[word]] +=1
    return result_vector

X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
print('X_train shape ', X_train_mybag.shape, '\nX_val shape ', X_val_mybag.shape)

X_train shape  (2, 10000) 
X_val shape  (2, 10000)


In [92]:
def tfidf_features(X_train, X_val, X_test):
    """
        X_train, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Create TF-IDF vectorizer with a proper parameters choice
    # Fit the vectorizer on the train set
    # Transform the train, test set and return the result
    
    
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=1.0, min_df=1, token_pattern='(\S+)')

    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_val_tfidf = tfidf_vectorizer.transform(X_val)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    
    return X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vectorizer.vocabulary_

X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}


In [95]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression


In [99]:
def train_classifier(X_train, y_train, C, regularisation):
    """
      X_train, y_train — training data
      
      return: trained classifier
    """
    
    # Create and fit LogisticRegression wraped into OneVsRestClassifier.

    model = OneVsRestClassifier(LogisticRegression(penalty=regularisation, C=C, max_iter=10000)).fit(X_train, y_train)
    return model

classifier_mybag = train_classifier(X_train_mybag, y_train, C = 4, regularisation = 'l2')
classifier_tfidf = train_classifier(X_train_tfidf, y_train, C = 4, regularisation = 'l2')

y_val_predicted_labels_mybag = classifier_mybag.predict(X_val_mybag)
y_val_predicted_labels_tfidf = classifier_tfidf.predict(X_val_tfidf)
y_val_predicted_scores_mybag = classifier_mybag.decision_function(X_val_mybag)
y_val_predicted_scores_tfidf = classifier_tfidf.decision_function(X_val_tfidf)

ValueError: Unknown label type: (0      3.1
1      3.1
2      7.4
3      7.5
4      4.2
5      3.1
6      7.1
7      3.1
8      3.1
9      8.3
10     7.2
11     7.7
12     7.2
13     7.3
14     4.3
15     8.1
16     7.6
17     8.9
18     NaN
19     8.2
20     4.3
21     7.4
22     8.2
23     8.1
24     3.1
25     8.7
26     5.4
27     8.3
28     7.2
29     8.1
      ... 
185    7.2
186    3.1
187    7.5
188    8.3
189    7.8
190    3.1
191    8.2
192    3.1
193    8.3
194    8.5
195    7.3
196    4.2
197    7.5
198    6.6
199    8.1
200    7.8
201    8.3
202    8.3
203    8.3
204    4.2
205    3.1
206    3.1
207    5.4
208    8.6
209    7.8
210    3.1
211    7.8
212    8.2
213    8.1
214    7.5
Name: Sub_markers, Length: 215, dtype: object,)

In [100]:
X_train

,Conversations,Free_format_Desc
0,· What do you/we want to aim for as an outcome...,youwe want aim outcome goal session
1,· What do we want to talk about to achieve as ...,want talk achieve outcome goal session
2,How will ......this help you to look at your s...,help look situation differently
3,How will ......this help you to look at yourr ...,help look yourr future life after achieve want
4,I hear your feeling ....... about what is happ...,hear feel happen
5,· What do you seek to accomplish as an outcome...,seek accomplish outcome goal session
6,What are you learning about yourself in this c...,learn conversation
7,· What do we desire to look forward to as an o...,desire look forward outcome goal session
8,· What do we want to discuss to aim for as an ...,want discuss aim outcome goal session
9,How will you use what have you learnt about yo...,use learn situation partner discusions far


In [101]:
y_train

0      3.1
1      3.1
2      7.4
3      7.5
4      4.2
5      3.1
6      7.1
7      3.1
8      3.1
9      8.3
10     7.2
11     7.7
12     7.2
13     7.3
14     4.3
15     8.1
16     7.6
17     8.9
18     NaN
19     8.2
20     4.3
21     7.4
22     8.2
23     8.1
24     3.1
25     8.7
26     5.4
27     8.3
28     7.2
29     8.1
      ... 
185    7.2
186    3.1
187    7.5
188    8.3
189    7.8
190    3.1
191    8.2
192    3.1
193    8.3
194    8.5
195    7.3
196    4.2
197    7.5
198    6.6
199    8.1
200    7.8
201    8.3
202    8.3
203    8.3
204    4.2
205    3.1
206    3.1
207    5.4
208    8.6
209    7.8
210    3.1
211    7.8
212    8.2
213    8.1
214    7.5
Name: Sub_markers, Length: 215, dtype: object